In [10]:
!pip install datasets transformers langchain rouge-score nltk google-cloud-aiplatform google-auth > /dev/null

In [23]:
!pip install bardapi > /dev/null

In [24]:
from bardapi import Bard
import os

os.environ['_BARD_API_KEY']="XAhWmJgg53Efs6JZkGfX536EA4SxveO-6w9wKkHAhjXDWWiien82o2RRQzjf6b2DZIws_A."


In [25]:
Bard()

In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS']="/content/generativeaitrial-trialLC.json"

## Load the dataset

In [2]:
import transformers
from datasets import load_dataset, load_metric

In [5]:
medium_datasets = load_dataset("Kamaljp/medium_articles")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/192368 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Kamaljp___parquet/Kamaljp--medium_articles-1c77e45b8e778fd3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset train/validation/test split

In [6]:
datasets_train_test = medium_datasets["train"].train_test_split(test_size=3000)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=3000)

medium_datasets["train"] = datasets_train_validation["train"]
medium_datasets["validation"] = datasets_train_validation["test"]
medium_datasets["test"] = datasets_train_test["test"]

medium_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
        num_rows: 186368
    })
    validation: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
        num_rows: 3000
    })
})

In [7]:
# keep only a subsample of the datasets
medium_datasets["train"] = medium_datasets["train"].shuffle().select(range(100000))
medium_datasets["validation"] = medium_datasets["validation"].shuffle().select(range(1000))
medium_datasets["test"] = medium_datasets["test"].shuffle().select(range(1000))

medium_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
        num_rows: 100000
    })
    validation: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
        num_rows: 1000
    })
})

In [ ]:
medium_datasets.push_to_hub("Kamaljp/articles_samples")

In [36]:
from langchain.llms import VertexAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = VertexAI(temperature=0)

# Define the prompt template for text-to-SQL conversion
prompt_template = PromptTemplate(
    input_variables=["article_body"],
    template="""Convert the above {article_body} to a table.
    Table must contain column of tags, explanation. 
    Explanation column contains 25 word explanation of tag.
    Then stop.""",
)

In [26]:
prompt_template.format(article_body='Hello')

'Convert the above Hello to a table.\n    Table must contain column of tags, explanation. \n    Explanation column contains 25 word explanation of tag'

In [8]:
test_text = medium_datasets['train'][0]['text']

In [37]:
data = Bard().get_answer(prompt_template.format(article_body=test_text))['content']

In [38]:
print(data)

Sure, here is a table of tags and explanations for blockchain technology:

| Tag | Explanation |
|---|---|
| **Blockchain** | A distributed ledger technology that allows for secure, transparent, and tamper-proof transactions. |
| **Immutability** | The property of a blockchain that makes it impossible to change or delete data once it has been added to the chain. |
| **Transparency** | The ability for anyone to view the complete history of transactions on a blockchain. |
| **Security** | The fact that blockchain is very difficult to hack or manipulate. |
| **Decentralization** | The lack of a central authority controlling a blockchain network. |
| **Peer-to-peer** | The ability for users to transact directly with each other on a blockchain network without the need for a third party. |
| **Scalability** | The ability for a blockchain network to handle increasing volume of transactions. |
| **Cost-efficiency** | The fact that blockchain can reduce costs associated with transactions and re

In [4]:
article_chain = LLMChain(llm=llm, prompt=prompt_template)

In [9]:
test_table = article_chain.run(test_text).strip()

In [10]:
print(test_table)

Tags | Explanation
------- | --------
Blockchain | A vast global distributed digital ledger that runs on millions of computers all over the world.
Bitcoin | A cryptocurrency that runs on this network.
Bitcoin mining | The process of creating new bitcoins by solving a complex cryptography problem.
Blockchain technology | An encrypted and distributed computing system that can be used for more than just cryptocurrencies.
Social Issues | Tenuous title claims, sharing economies, remittances, identity ownership, the democratization of data access, online voting, and financial markets.
Blockchain can be used to provide immutable land titles. | This would help give millions of people


In [44]:
import time
def get_table(line_item):
  text = line_item['text']
  time.sleep(5) #better to slow down the API calls
  retry = 0
  #creating retry loops
  while True:

    try:
      line_item['tabled_format']=article_chain.run(text).strip()

    except Exception as e:
      print(e)
      print("Going to sleep for 60 sec")
      time.sleep(60)
      retry += 1
      print(f"Retrying {retry}")

      if retry >= 3:
        raise KeyboardInterrupt
        
  return line_item

In [47]:
import time
def get_df_table(text):
  time.sleep(5) #better to slow down the API calls
  retry = 0
  #creating retry loops
  while True:

    try:
      tabled_format = article_chain.run(text).strip()

    except Exception as e:
      print(e)
      print("Going to sleep for 60 sec")
      time.sleep(60)
      retry += 1
      print(f"Retrying {retry}")

      if retry >= 3:
        raise KeyboardInterrupt
        
  return tabled_format

In [39]:
import time
def get_table_b(line_item):
  text = line_item['text']
  time.sleep(5) #better to slow down the API calls
  retry = 0
  #creating retry loops
  while True:

    try:
      line_item['tabled_format']= Bard().get_answer(prompt_template.format(article_body=text))['content']

    except Exception as e:
      print(e)
      print("Going to sleep for 60 sec")
      time.sleep(60)
      retry += 1
      print(f"Retrying {retry}")

      if retry >= 3:
        line_item['tabled_format'] = 'Unable to fetch tabled format.'
        raise KeyboardInterrupt
        
  return line_item

In [14]:
article_1000_ds = medium_datasets['train'].shuffle().select(range(1000))

In [41]:
article_df = article_1000_ds.to_pandas()

In [42]:
article_df.head()

,title,text,url,authors,timestamp,tags
0,"Can we have joy, too?","Ocean Beach, San Francisco\n\n“Oftentimes when...",https://medium.com/urban-empath/can-we-have-jo...,['Tal Lee Anderman'],2020-10-05 03:19:41.661000+00:00,"['Joy', 'Coaching', 'Feelings', 'Highly Sensit..."
1,Inner Machinations: Entry_023.,Inner Machinations: Entry_023.\n\nToday was th...,https://medium.com/@elmohidalgo/inner-machinat...,['Elmo Hidalgo'],2020-12-25 09:24:30.939000+00:00,"['Games', 'Journaling', 'Life', 'Pokemon', 'Pr..."
2,7 Entrepreneurial traps you should avoid right...,Being an entrepreneur has several pitfalls; it...,https://medium.com/startupwriter/7-entrepreneu...,['Midhun Areeckal'],2020-12-27 13:30:22.773000+00:00,"['Entrepreneurship', 'Business Risk', 'Entrepr..."
3,Marketing and The Psychology of Colors — Mobiteam,The psychology of color as it relates to persu...,https://medium.com/@sobieski/marketing-and-the...,['Marcel Sobieski'],2021-01-25 08:58:31.014000+00:00,"['Digital Marketing', 'Wordpress Web Developme..."
4,Your Thoughts Create Your World,"Our thoughts create our circumstances, so we e...",https://medium.com/spiritual-secrets/your-thou...,['Lanu Pitan'],2020-11-14 07:02:35.374000+00:00,"['Spirituality', 'Poetry', 'Poetry On Medium',..."


In [ ]:
article_df.loc[:,'tabled_format'] = article_df.text.apply(lambda x: get_df_table(x))

In [ ]:
print(article_table['tabled_format'][9])

Tags | Explanation
------- | --------
Self-talk | Talking to yourself more positively and with kindness.
Get out of your comfort zone | Doing activities that you fear and run away from.
Create new habits | Starting something new.
Ask people for help | Discussing your problems with others.
Fake it till you make it | Believing in yourself even when you don't feel like it.


In [ ]:
article_table.push_to_hub("Kamaljp/article_w_table")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
article_samples = medium_datasets.map(get_table)

In [ ]:
article_samples

## Data preprocessing

In [ ]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

In [ ]:
model_checkpoint = "t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
prefix = "summarize: "

max_input_length = 512
max_target_length = 64

def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["text"]]
  inputs = [prefix + text for text in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["title"], max_length=max_target_length, 
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
medium_datasets_cleaned = medium_datasets.filter(lambda example: (len(example['text']) >= 500) and (len(example['title']) >= 20))
tokenized_datasets = medium_datasets_cleaned.map(preprocess_data, batched=True)
tokenized_datasets

## Fine-tune T5

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
!rm -r {model_dir}

In [ ]:
batch_size = 8
model_name = "t5-base-medium-title-generation"
model_dir = f"drive/MyDrive/Models/{model_name}"
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
import numpy as np

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Start TensorBoard before training to monitor it in progress
%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

## Load the model from GDrive

In [ ]:
model_name = "t5-base-medium-title-generation/checkpoint-2000"
model_dir = f"drive/MyDrive/Models/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [ ]:
text = """
We define access to a Streamlit app in a browser tab as a session.
For each browser tab that connects to the Streamlit server, a new session is created.
Streamlit reruns your script from top to bottom every time you interact with your app.
Each reruns takes place in a blank slate: no variables are shared between runs.
Session State is a way to share variables between reruns, for each user session.
In addition to the ability to store and persist state, Streamlit also exposes the
ability to manipulate state using Callbacks. In this guide, we will illustrate the
usage of Session State and Callbacks as we build a stateful Counter app.
For details on the Session State and Callbacks API, please refer to our Session
State API Reference Guide. Also, check out this Session State basics tutorial
video by Streamlit Developer Advocate Dr. Marisa Smith to get started:
"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Session State and Callbacks in Streamlit

In [ ]:
text = """
Many financial institutions started building conversational AI, prior to the Covid19
pandemic, as part of a digital transformation initiative. These initial solutions
were high profile, highly personalized virtual assistants — like the Erica chatbot
from Bank of America. As the pandemic hit, the need changed as contact centers were
under increased pressures. As Cathal McGloin of ServisBOT explains in “how it started,
and how it is going,” financial institutions were looking for ways to automate
solutions to help get back to “normal” levels of customer service. This resulted
in a change from the “future of conversational AI” to a real tactical assistant
that can help in customer service. Haritha Dev of Wells Fargo, saw a similar trend.
Banks were originally looking to conversational AI as part of digital transformation
to keep up with the times. However, with the pandemic, it has been more about
customer retention and customer satisfaction. In addition, new use cases came about
as a result of Covid-19 that accelerated adoption of conversational AI. As Vinita
Kumar of Deloitte points out, banks were dealing with an influx of calls about new
concerns, like questions around the Paycheck Protection Program (PPP) loans. This
resulted in an increase in volume, without enough agents to assist customers, and
tipped the scale to incorporate conversational AI. When choosing initial use cases
to support, financial institutions often start with high volume, low complexity
tasks. For example, password resets, checking account balances, or checking the
status of a transaction, as Vinita points out. From there, the use cases can evolve
as the banks get more mature in developing conversational AI, and as the customers
become more engaged with the solutions. Cathal indicates another good way for banks
to start is looking at use cases that are a pain point, and also do not require a
lot of IT support. Some financial institutions may have a multi-year technology
roadmap, which can make it harder to get a new service started. A simple chatbot
for document collection in an onboarding process can result in high engagement,
and a high return on investment. For example, Cathal has a banking customer that
implemented a chatbot to capture a driver’s license to be used in the verification
process of adding an additional user to an account — it has over 85% engagement
with high satisfaction. An interesting use case Haritha discovered involved
educating customers on financial matters. People feel more comfortable asking a
chatbot what might be considered a “dumb” question, as the chatbot is less judgmental.
Users can be more ambiguous with their questions as well, not knowing the right
words to use, as chatbot can help narrow things down.
"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Conversational AI: The Future of Customer Service

## Upload the model to the Hugging Space Hub

https://huggingface.co/docs/transformers/model_sharing

In [ ]:
!pip install --upgrade jax jaxlib # CPU-only
!pip install flax

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration, FlaxT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained(model_dir)
pt_model = T5ForConditionalGeneration.from_pretrained(model_dir)
tf_model = TFT5ForConditionalGeneration.from_pretrained(model_dir, from_pt=True)
flax_model = FlaxT5ForConditionalGeneration.from_pretrained(model_dir, from_pt=True)

tokenizer.push_to_hub(model_name)
pt_model.push_to_hub(model_name)
tf_model.push_to_hub(model_name)
flax_model.push_to_hub(model_name)

## Load the model from the Hugging Face Hub

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

tokenizer = AutoTokenizer.from_pretrained("fabiochiu/t5-small-medium-title-generation")
model = AutoModelForSeq2SeqLM.from_pretrained("fabiochiu/t5-small-medium-title-generation")

text = """
Many financial institutions started building conversational AI, prior to the Covid19
pandemic, as part of a digital transformation initiative. These initial solutions
were high profile, highly personalized virtual assistants — like the Erica chatbot
from Bank of America. As the pandemic hit, the need changed as contact centers were
under increased pressures. As Cathal McGloin of ServisBOT explains in “how it started,
and how it is going,” financial institutions were looking for ways to automate
solutions to help get back to “normal” levels of customer service. This resulted
in a change from the “future of conversational AI” to a real tactical assistant
that can help in customer service. Haritha Dev of Wells Fargo, saw a similar trend.
Banks were originally looking to conversational AI as part of digital transformation
to keep up with the times. However, with the pandemic, it has been more about
customer retention and customer satisfaction. In addition, new use cases came about
as a result of Covid-19 that accelerated adoption of conversational AI. As Vinita
Kumar of Deloitte points out, banks were dealing with an influx of calls about new
concerns, like questions around the Paycheck Protection Program (PPP) loans. This
resulted in an increase in volume, without enough agents to assist customers, and
tipped the scale to incorporate conversational AI. When choosing initial use cases
to support, financial institutions often start with high volume, low complexity
tasks. For example, password resets, checking account balances, or checking the
status of a transaction, as Vinita points out. From there, the use cases can evolve
as the banks get more mature in developing conversational AI, and as the customers
become more engaged with the solutions. Cathal indicates another good way for banks
to start is looking at use cases that are a pain point, and also do not require a
lot of IT support. Some financial institutions may have a multi-year technology
roadmap, which can make it harder to get a new service started. A simple chatbot
for document collection in an onboarding process can result in high engagement,
and a high return on investment. For example, Cathal has a banking customer that
implemented a chatbot to capture a driver’s license to be used in the verification
process of adding an additional user to an account — it has over 85% engagement
with high satisfaction. An interesting use case Haritha discovered involved
educating customers on financial matters. People feel more comfortable asking a
chatbot what might be considered a “dumb” question, as the chatbot is less judgmental.
Users can be more ambiguous with their questions as well, not knowing the right
words to use, as chatbot can help narrow things down.
"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Conversational AI: The Future of Customer Service

## Evaluate the model on the test set

In [ ]:
import torch

# get test split
test_tokenized_dataset = tokenized_datasets["test"]

# pad texts to the same length
def preprocess_test(examples):
  inputs = [prefix + text for text in examples["text"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                           padding="max_length")
  return model_inputs

test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=32)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(test_tokenized_dataset["title"], max_length=max_target_length,
                       truncation=True, padding="max_length")["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)